In [4]:
# Load libraries
import gurobipy as gp
from gurobipy import GRB,tuplelist
import numpy as np
import pandas as pd

In [5]:
# Set parameters
P = 0
p = -5   # Defining the accuracy

def optimize_problem(p,P):
    set_l = range(p,P+1)
    set_k = range(10)
    set_kl = tuplelist([(k,l) for l in set_l for k in set_k])

    m = gp.Model('MDT')

    # Define the continuous variables
    x1 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x1")
    x2 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x2")

    m.update()

    w12 = m.addVar(vtype=GRB.CONTINUOUS, name="w12")
    Delta_w12 = m.addVar(lb=0, ub=1.5 * x1.UB, vtype=GRB.CONTINUOUS, name="Delta_w12")  # 1.5 * 1.5 upper bound
    Delta_x1 = m.addVar(lb=0, ub=10**p, vtype=GRB.CONTINUOUS, name="Delta_x1")

    # Define the indexed continuous variables (hat_x_k) and binary variables (z_k)
    hat_x = m.addVars(set_kl, lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="hat_x")
    z = m.addVars(set_kl, vtype=GRB.BINARY, name="z")

    # Set the objective function: Minimize f = -x1 + w12 - x2
    m.setObjective(-x1 + w12 - x2, GRB.MINIMIZE)
    m.update()

    # Constraint 1: -6*x1 + 8*x2 <= 3
    m.addConstr(-6*x1 + 8*x2 <= 3, "c1")

    # Constraint 2: 3*x1 - x2 <= 3
    m.addConstr(3*x1 - x2 <= 3, "c2")

    # Constraint 3: w12 = sum(k * hat_x[k]) + Delta_w12
    m.addConstr(w12 == gp.quicksum(gp.quicksum(10**l * k * hat_x[k,l] for k in set_k) for l in set_l) + Delta_w12, "c3")

    # Constraint 4: x1 = sum(10^0 * k * z_k) + Delta_x1
    m.addConstr(x1 == gp.quicksum(gp.quicksum(10**l * k * z[k,l] for k in set_k) for l in set_l) + Delta_x1, "c4")

    # Constraint 5: x2 = sum(hat_x[k])
    m.addConstrs((x2 == gp.quicksum(hat_x[k,l] for k in set_k) for l in set_l), "c5")

    # Constraint 6: hat_x[k] <= 1.5 * z[k] for all k
    # for k in range(10):
    m.addConstrs((hat_x[k,l] <= 1.5 * z[k,l] for k,l in set_kl), "c6")

    # Constraint 7: Sum(z_k) = 1
    m.addConstrs((gp.quicksum(z[k,l] for k in set_k) == 1 for l in set_l), "c7")

    # Constraint 8: 0 <= Delta_w12 <= 1.5 * Delta_x1
    m.addConstr(Delta_w12 >= 0, "c8a")
    m.addConstr(Delta_w12 <= 1.5 * Delta_x1, "c8b")

    # Constraint 9: (x2 - 1.5) * 10^0 + 1.5 * Delta_x1 <= Delta_w12 <= x2 * 10^0
    m.addConstr((x2 - 1.5) * 10**p + 1.5 * Delta_x1 <= Delta_w12, "c9a")
    m.addConstr(Delta_w12 <= x2 * 10**p, "c9b")

    # Constraint 10: 0 <= Delta_x1 <= 1.5 * 10^0
    m.addConstr(Delta_x1 >= 0, "c10a")
    m.addConstr(Delta_x1 <= 1.5 * 10**p, "c10b")

    # Optimize the m
    m.Params.OutputFlag = 0
    # m.write('kk.lp')
    m.optimize()

    # print('-------------- p =',p,'----------------')
    # # print('There are', m.NumVars, 'variables and', m.NumConstrs, 'constraints')
    # print('The program runs for', m.Runtime, 'seconds')
    # print('Objective value:',m.ObjVal,'; x1=',x1.X,'; x2=',x2.X,'; w12=', w12.X)
    # print(' ')

    return m.ObjVal, x1.X, x2.X, m.Runtime, m.NumVars, m.NumConstrs

In [6]:
df = pd.DataFrame(columns=['p','Obj_value','x1','x2','Computed in'])
for i in range(0,-7,-1):
    objval,x1,x2,runtime,numvars,numconstr = optimize_problem(i,0)
    df2 = pd.DataFrame({'p':[i], 'Obj_value':[objval], 'x1':[x1], 'x2':[x2], 'Computed in':[runtime], 'Num. Vars':[numvars], 'Num. Constrs':[numconstr]},index=[i])
    df = pd.concat([df,df2])

df.set_index(['p'])
print(df)

     p  Obj_value        x1        x2  Computed in  Num. Vars  Num. Constrs
 0   0  -1.333333  1.333333  1.000000        0.004       25.0          22.0
-1  -1  -1.116667  1.166667  0.500000        0.008       45.0          34.0
-2  -2  -1.086667  1.166667  0.500000        0.017       65.0          46.0
-3  -3  -1.083667  1.166667  0.500000        0.038       85.0          58.0
-4  -4  -1.083367  1.166667  0.500000        0.063      105.0          70.0
-5  -5  -1.083337  1.166667  0.500000        0.080      125.0          82.0
-6  -6  -1.083334  1.166789  0.500364        0.098      145.0          94.0


In [9]:
x1

1.166789